# ActBlue donor profile

In [7]:
import json
import numpy as np
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import os

In [8]:
%matplotlib inline

plt.style.use('ggplot')

pd.options.display.float_format = '{:,.2f}'.format

In [9]:
def read_or_save(name, func):
    path = 'pickles/' + name + '.pickle.gz'
    if (os.path.isfile(path)):
        return pd.read_pickle(path)
    else:
        result = func()
        os.makedirs('pickles', exist_ok=True)
        result.to_pickle(path)
        return result

In [10]:
committee_id = "C00401224"
year_since = 2013

### Connect to the PostgreSQL database

In [11]:
with open("config.json") as f:
    conf = json.load(f)
conn = psycopg2.connect(
    dbname=conf['dbname'],
    user=conf['user'],
    host=conf['host'],
    password=conf['password']
)

### Where do the donors giving through ActBlue come from geographically? Any striking or interesting patterns, or zip codes that typically don’t contribute?

In [ ]:
actblue_states = read_or_save(
    'actblue_states',
    lambda: pd.read_sql("""
    SELECT contributor_state,
           count(*),
           sum(contribution_amount)
    FROM fec_contributions
    WHERE filing_id IN
        (SELECT filing_id
         FROM fec_pac_summaries
         JOIN fec_amended_filings USING (filing_id)
         WHERE filer_committee_id_number = '""" + committee_id + """'
           AND extract(YEAR
                       FROM coverage_through_date) > """ + str(year_since) + """
         ORDER BY coverage_through_date DESC)
      AND form_type = 'SA11AI'
    GROUP BY contributor_state
    """, con=conn)
)
actblue_states.sort_values(by=['sum'], ascending=False)

In [ ]:
clinton_votes = pd.read_csv('clintonvotes.csv')

clinton_votes

votes_vs_contribs = actblue_states.merge(clinton_votes, left_on='contributor_state', right_on='postal', how='inner')

votes_vs_contribs['dollars_per'] = votes_vs_contribs['sum']/votes_vs_contribs['votes']

votes_vs_contribs.sort_values(by=['dollars_per'], ascending=False)

In [ ]:
votes_vs_contribs['contribs_per'] = votes_vs_contribs['count']/votes_vs_contribs['votes']

votes_vs_contribs.sort_values(by=['contribs_per'], ascending=False)

### Questions to answer
> - Where do the donors giving through ActBlue come from geographically? Any striking or interesting patterns, or zip codes that typically don’t contribute?
> - Where is the money going? Are people giving to local candidates or are they giving to candidates around the country? Which campaigns/candidates/groups have done the best job tapping into this network?
> - What’s the gender breakdown and how does it compare to the gender breakdown of itemized contributions? Is it different?
> - Are these new donors?
> - Can we tell if these are recurring contributions or one time contributions?
> - Do the majority of these donors give repeatedly or are they one and done? How does that compare to itemized contributions? How “sticky” is this?


In [ ]:
actblue_in_state = read_or_save(
    'actblue_in_state',
    lambda: pd.read_sql("""
    SELECT CASE
               WHEN beneficiary_candidate_state = contributor_state THEN TRUE
               ELSE FALSE
           END AS in_state,
           sum(contribution_amount),
           count(*)
    FROM fec_contributions
    JOIN fec_expenditures ON fec_contributions.filing_id IN
      (SELECT filing_id
       FROM fec_pac_summaries
       JOIN fec_amended_filings USING (filing_id)
       WHERE filer_committee_id_number = '""" + committee_id + """'
         AND extract(YEAR
                     FROM coverage_through_date) > 2016
       ORDER BY coverage_through_date DESC)
    AND fec_expenditures.filing_id = fec_contributions.filing_id
    AND fec_contributions.form_type = 'SA11AI'
    AND fec_expenditures.form_type = 'SB23'
    AND replace(transaction_id_number,'SB23_','') = replace(transaction_id,'SA11AI_','')
    AND beneficiary_candidate_state IS NOT NULL
    GROUP BY in_state
    """, con=conn)
)
actblue_in_state

In [ ]:
actblue_freq = read_or_save(
    'actblue_freq',
    lambda: pd.read_sql("""
    SELECT COUNT,
           count(*) AS count_of_count
    FROM
      (SELECT contributor_first_name,
              contributor_last_name,
              left(contributor_zip_code,5),
              count(*) AS COUNT
       FROM fec_contributions
       WHERE filing_id IN
           (SELECT filing_id
            FROM fec_pac_summaries
            JOIN fec_amended_filings USING (filing_id)
            WHERE filer_committee_id_number = '""" + committee_id + """'
              AND extract(YEAR
                          FROM coverage_through_date) > 2016
            ORDER BY coverage_through_date DESC)
         AND form_type = 'SA11AI'
       GROUP BY contributor_first_name,
                contributor_last_name,
                left(contributor_zip_code,5)) AS donors
    GROUP BY COUNT
    """, con=conn)
)
actblue_freq

In [ ]:
actblue_addicts = read_or_save(
    'actblue_addicts',
    lambda: pd.read_sql("""
    SELECT COUNT,
           count(*) AS count_of_count,
           array_agg(DISTINCT contributor_organization_name),
           array_agg(DISTINCT contributor_first_name),
           array_agg(DISTINCT contributor_last_name),
           array_agg(DISTINCT contributor_zip_code)
    FROM
      (SELECT contributor_organization_name,
              contributor_first_name,
              contributor_last_name,
              left(contributor_zip_code,5) as contributor_zip_code,
              count(*) AS COUNT
       FROM fec_contributions
       WHERE filing_id IN
           (SELECT filing_id
            FROM fec_pac_summaries
            JOIN fec_amended_filings USING (filing_id)
            WHERE filer_committee_id_number = '""" + committee_id + """'
              AND extract(YEAR
                          FROM coverage_through_date) > 2016
            ORDER BY coverage_through_date DESC)
         AND form_type = 'SA11AI'
       GROUP BY contributor_organization_name,
                contributor_first_name,
                contributor_last_name,
                left(contributor_zip_code,5)) AS donors
    GROUP BY COUNT
    HAVING count(*) <= 3
    ORDER BY COUNT DESC
    LIMIT 30
    """, con=conn)
)
actblue_addicts

